# Ch 4 of Montgomery and Peck

In [8]:
%pylab inline
matplotlib.rcParams['figure.figsize'] = (12, 10)
import pandas as pd
import statsmodels.formula.api as sm
from scipy import stats
from patsy import dmatrices

Populating the interactive namespace from numpy and matplotlib


## Summary of equations

Equation $\boldsymbol{y = X \beta + \varepsilon}$, where $\boldsymbol X$ is (n, k),  $\boldsymbol \beta$ is (k, 1), and $\boldsymbol y$, $\boldsymbol \varepsilon$ are (n,1).

We are minimizing the sum of squares $S(\boldsymbol{\beta}) = \sum_{i=1}^n \varepsilon_i^2 = \boldsymbol{\varepsilon' \varepsilon} = \boldsymbol{(y - X\beta)'(y - X\beta)} = \boldsymbol{y'y - 2\beta'X'y + \beta'X'X\beta}$. Taking derivative w.r.t. $\boldsymbol\beta$ we get $\boldsymbol{X'X\hat\beta = X'y}$, where we replaced $\boldsymbol\beta$ with $\boldsymbol {\hat\beta}$ to represent that, at the point of minimal S, the estimator matches the true model.

The solution of the normal equation (above) is $\boldsymbol{\hat\beta} = \boldsymbol{(X'X)^{-1}X'y}$, provided that $\boldsymbol{(X'X)^{-1}}$ exists. This is the case if the regressors are linearly independent.

We can get the fitted values for the observed quantities $\boldsymbol{\hat y} = \boldsymbol{X\hat\beta} = \boldsymbol{X(X'X)^{-1}X'y} = \boldsymbol{Hy}$. $\boldsymbol{H}$ is called the hat matrix and it maps the vector of observed values to a vector of fitted values.

In [13]:
%%file delivery-time-data.csv
"Observation number", "Delivery time (minutes)", "Number of cases", "Distance (feet)"
1, 16.68, 7, 560
2, 11.50, 3, 220
3, 12.05, 3, 340
4, 14.88, 4, 80
5, 13.75, 6, 150
6, 18.11, 7, 330
7, 8.00, 2, 110
8, 17.83, 7, 210
9, 79.24, 30, 1460
10, 21.50, 5, 605
11, 40.33, 16, 688
12, 21.00, 10, 215
13, 13.50, 4, 255
14, 19.75, 6, 462
15, 24.00, 9, 448
16, 29.00, 10, 776
17, 15.35, 6, 200
18, 19.00, 7, 132
19, 9.50, 3, 36
20, 35.10, 17, 770
21, 17.90, 10, 140
22, 52.32, 26, 810
23, 18.75, 9, 450
24, 19.83, 8, 635
25, 10.75, 4, 150

Overwriting delivery-time-data.csv


In [14]:
dt = pd.read_csv('delivery-time-data.csv', index_col='Observation number')
dt.columns = ['t', 'n', 'd'] # have to rename columns to use formulas

In [15]:
dt

,t,n,d
Observation number,,,
1,16.68,7,560
2,11.50,3,220
3,12.05,3,340
4,14.88,4,80
5,13.75,6,150
6,18.11,7,330
7,8.00,2,110
8,17.83,7,210
9,79.24,30,1460


In [21]:
y, X = dmatrices('t ~ n + d', data=dt)

DesignMatrix with shape (25, 3)
  Intercept   n     d
          1   7   560
          1   3   220
          1   3   340
          1   4    80
          1   6   150
          1   7   330
          1   2   110
          1   7   210
          1  30  1460
          1   5   605
          1  16   688
          1  10   215
          1   4   255
          1   6   462
          1   9   448
          1  10   776
          1   6   200
          1   7   132
          1   3    36
          1  17   770
          1  10   140
          1  26   810
          1   9   450
          1   8   635
          1   4   150
  Terms:
    'Intercept' (column 0)
    'n' (column 1)
    'd' (column 2)